<a href="https://colab.research.google.com/github/Talizg03/weather/blob/main/4_2_25ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_csv("Train.csv")

<ipython-input-1-97fbf718528a>:2: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Train.csv")


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126919 entries, 0 to 126918
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   126919 non-null  int64  
 1   SalePrice                 126919 non-null  int64  
 2   MachineID                 126919 non-null  int64  
 3   ModelID                   126919 non-null  int64  
 4   datasource                126919 non-null  int64  
 5   auctioneerID              126919 non-null  int64  
 6   YearMade                  126919 non-null  int64  
 7   MachineHoursCurrentMeter  26735 non-null   float64
 8   UsageBand                 23809 non-null   object 
 9   saledate                  126919 non-null  object 
 10  fiModelDesc               126919 non-null  object 
 11  fiBaseModel               126919 non-null  object 
 12  fiSecondaryDesc           90484 non-null   object 
 13  fiModelSeries             9986 non-null    o

In [3]:
df2=df.copy()
df2.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Convert 'date_of_sale' to datetime format
df2['date_of_sale'] = pd.to_datetime(df2['saledate'], format='%m/%d/%Y %H:%M', errors='coerce')

# Add AgeOfVehicle column (date of sale year - YearMade)
df2['AgeOfVehicle'] = df2['date_of_sale'].dt.year - df2['YearMade']

# Display the first few rows of the updated DataFrame

df2.head()





,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,date_of_sale,AgeOfVehicle
0,1139246,66000,999089,3157,121,3,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,2006-11-16,2
1,1139248,57000,117657,77,121,3,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,2004-03-26,8
2,1139249,10000,434808,7009,121,3,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004-02-26,3
3,1139251,38500,1026470,332,121,3,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-05-19,10
4,1139253,11000,1057373,17311,121,3,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-07-23,2


In [5]:
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
import pandas as pd

# turn all string/category columns into integers

for col in df2.select_dtypes(['object']):
    df2[col] = df[col].astype('category')
df2.info()

for col in df2.select_dtypes(['category']):
    df2[col] = df2[col].cat.codes

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126919 entries, 0 to 126918
Data columns (total 55 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   126919 non-null  int64         
 1   SalePrice                 126919 non-null  int64         
 2   MachineID                 126919 non-null  int64         
 3   ModelID                   126919 non-null  int64         
 4   datasource                126919 non-null  int64         
 5   auctioneerID              126919 non-null  int64         
 6   YearMade                  126919 non-null  int64         
 7   MachineHoursCurrentMeter  26735 non-null   float64       
 8   UsageBand                 23809 non-null   category      
 9   saledate                  126919 non-null  category      
 10  fiModelDesc               126919 non-null  category      
 11  fiBaseModel               126919 non-null  category      
 12  fi

In [6]:
# create X and y
X = df2.drop(columns='SalePrice')
display(X.head())
y = df2['SalePrice']
display(y)

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,date_of_sale,AgeOfVehicle
0,1139246,999089,3157,121,3,2004,68.0,1,588,586,...,-1,-1,-1,-1,-1,-1,2,1,2006-11-16,2
1,1139248,117657,77,121,3,1996,4640.0,1,1568,1059,...,-1,-1,-1,-1,-1,-1,2,1,2004-03-26,8
2,1139249,434808,7009,121,3,2001,2838.0,0,1248,216,...,-1,-1,-1,-1,-1,-1,-1,-1,2004-02-26,3
3,1139251,1026470,332,121,3,2001,3486.0,0,2183,1949,...,-1,-1,-1,-1,-1,-1,-1,-1,2011-05-19,10
4,1139253,1057373,17311,121,3,2007,722.0,2,2901,2179,...,-1,-1,-1,-1,-1,-1,-1,-1,2009-07-23,2


,SalePrice
0,66000
1,57000
2,10000
3,38500
4,11000
...,...
126914,16000
126915,13000
126916,15250
126917,8500


In [7]:
# split into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.7)
X_train

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,date_of_sale,AgeOfVehicle
73808,1360592,775605,23929,132,7,1995,NaN,-1,3322,62,...,-1,-1,-1,-1,-1,-1,-1,-1,2000-09-15,5
14452,1186701,159104,1580,121,3,2005,2709.0,2,127,1328,...,-1,-1,-1,0,4,5,-1,-1,2009-01-29,4
82730,1381832,1532894,4203,132,23,1990,NaN,-1,373,1472,...,2,1,2,-1,-1,-1,-1,-1,1995-10-28,5
11909,1178082,1036811,658,121,3,2004,919.0,1,1486,2497,...,-1,-1,-1,-1,-1,-1,-1,-1,2008-03-20,4
29775,1261117,1509012,6788,132,18,1979,NaN,-1,1890,643,...,-1,-1,-1,-1,-1,-1,-1,-1,1992-04-25,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,1461367,532771,7113,132,2,1987,NaN,-1,1179,477,...,-1,-1,-1,-1,-1,-1,-1,-1,2003-02-02,16
103694,1425773,1420238,4154,132,6,1972,NaN,-1,2193,1441,...,-1,-1,-1,0,6,5,-1,-1,1997-05-20,25
860,1142111,1035047,4093,121,3,2003,2750.0,0,446,1272,...,-1,-1,-1,0,4,5,-1,-1,2006-10-05,3
15795,1191604,1069513,20087,121,3,2002,0.0,-1,2422,2305,...,2,1,0,-1,-1,-1,-1,-1,2009-06-12,7


In [8]:
# print the RMSE accuracy of the baseline (std dev)
print("RMSE Baseline accuracy:", y_test.std())

RMSE Baseline accuracy: 23487.492770595687


In [9]:
# create X and y
X = df2.drop(columns=['SalePrice','date_of_sale'])
display(X.head())
y = df2['SalePrice']
display(y)

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,AgeOfVehicle
0,1139246,999089,3157,121,3,2004,68.0,1,588,586,...,-1,-1,-1,-1,-1,-1,-1,2,1,2
1,1139248,117657,77,121,3,1996,4640.0,1,1568,1059,...,-1,-1,-1,-1,-1,-1,-1,2,1,8
2,1139249,434808,7009,121,3,2001,2838.0,0,1248,216,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,3
3,1139251,1026470,332,121,3,2001,3486.0,0,2183,1949,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,10
4,1139253,1057373,17311,121,3,2007,722.0,2,2901,2179,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,2


,SalePrice
0,66000
1,57000
2,10000
3,38500
4,11000
...,...
126914,16000
126915,13000
126916,15250
126917,8500


In [10]:
# split into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.7)
X_train

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,AgeOfVehicle
73808,1360592,775605,23929,132,7,1995,NaN,-1,3322,62,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,5
14452,1186701,159104,1580,121,3,2005,2709.0,2,127,1328,...,-1,-1,-1,-1,0,4,5,-1,-1,4
82730,1381832,1532894,4203,132,23,1990,NaN,-1,373,1472,...,23,2,1,2,-1,-1,-1,-1,-1,5
11909,1178082,1036811,658,121,3,2004,919.0,1,1486,2497,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,4
29775,1261117,1509012,6788,132,18,1979,NaN,-1,1890,643,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,1461367,532771,7113,132,2,1987,NaN,-1,1179,477,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,16
103694,1425773,1420238,4154,132,6,1972,NaN,-1,2193,1441,...,-1,-1,-1,-1,0,6,5,-1,-1,25
860,1142111,1035047,4093,121,3,2003,2750.0,0,446,1272,...,-1,-1,-1,-1,0,4,5,-1,-1,3
15795,1191604,1069513,20087,121,3,2002,0.0,-1,2422,2305,...,23,2,1,0,-1,-1,-1,-1,-1,7


In [11]:
# print the RMSE accuracy of the baseline (std dev)
print("RMSE Baseline accuracy:", y_test.std())

RMSE Baseline accuracy: 23487.492770595687


In [12]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100,random_state=42, n_jobs=-1)
model = model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

def RMSE(y_, y_pred_):
    return ((y_ - y_pred_) ** 2).mean() ** 0.5

print(f'Train RMSE: {RMSE(y_train, y_train_pred):.3f}')
print(f'Test RMSE: {RMSE(y_test, y_test_pred):.3f}')

Train RMSE: 2793.092
Test RMSE: 7309.590


In [23]:
# Train the Isolation Forest model
from sklearn.ensemble import IsolationForest
iso_forest = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
iso_forest.fit(X_train)

# Predict anomaly scores and labels on a copy of X_test
X_test_copy = X_test.copy()
df2['anomaly_score'] = iso_forest.decision_function(X)  # Quantitative weirdness
df2['anomaly'] = iso_forest.predict(X)  # Binary anomaly label
df2['anomaly_label'] = df2['anomaly'].map({1: 'Normal', -1: 'Anomaly'})

In [24]:
df2.sort_values("anomaly_score").sample(20)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,date_of_sale,AgeOfVehicle,anomaly_score,anomaly,anomaly_label
108271,1439260,82000,681715,3807,132,4,2001,NaN,-1,1315,...,-1,-1,-1,2,1,2005-02-07,4,0.140353,1,Normal
37113,1274843,12500,1089708,3217,132,1,2003,NaN,-1,3327,...,-1,-1,-1,-1,-1,2006-09-15,3,0.111169,1,Normal
79050,1373260,10000,1507390,6979,132,1,1972,NaN,-1,1020,...,-1,-1,-1,-1,-1,1996-12-05,24,0.030718,1,Normal
115953,1453134,21000,1147850,3854,132,8,1978,NaN,-1,3428,...,-1,-1,-1,2,1,2005-09-22,27,0.132193,1,Normal
102570,1422961,10000,1250190,3877,132,1,1987,NaN,-1,2027,...,-1,-1,-1,2,1,1995-04-07,8,0.158305,1,Normal
98581,1415970,43000,1150807,3357,132,7,1977,NaN,-1,3322,...,-1,-1,-1,-1,-1,2000-09-15,23,0.044936,1,Normal
3112,1149663,113500,510838,23931,121,3,2004,6768.0,0,1723,...,-1,-1,-1,-1,-1,2009-04-10,5,-0.013498,-1,Anomaly
29014,1260091,11000,1405091,6788,132,10,1979,NaN,-1,2205,...,-1,-1,-1,-1,-1,1993-05-21,14,0.164894,1,Normal
119887,1461376,19000,1334398,7113,132,2,1987,NaN,-1,74,...,-1,-1,-1,-1,-1,1993-01-23,6,0.129851,1,Normal
80240,1376178,25000,1494291,107,132,10,1993,NaN,-1,1627,...,0,5,5,-1,-1,2000-03-30,7,0.149855,1,Normal


In [25]:
df2.anomaly.value_counts()

,count
anomaly,
1,125664
-1,1255


In [26]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126919 entries, 0 to 126918
Data columns (total 58 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   126919 non-null  int64         
 1   SalePrice                 126919 non-null  int64         
 2   MachineID                 126919 non-null  int64         
 3   ModelID                   126919 non-null  int64         
 4   datasource                126919 non-null  int64         
 5   auctioneerID              126919 non-null  int64         
 6   YearMade                  126919 non-null  int64         
 7   MachineHoursCurrentMeter  26735 non-null   float64       
 8   UsageBand                 126919 non-null  int8          
 9   saledate                  126919 non-null  int16         
 10  fiModelDesc               126919 non-null  int16         
 11  fiBaseModel               126919 non-null  int16         
 12  fi

In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

model = RandomForestRegressor()
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print("Test RMSE:", mean_squared_error(y_test, y_test_pred) ** 0.5)

Test RMSE: 7306.595675706572


In [28]:
df3=df2.copy()
df3.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,date_of_sale,AgeOfVehicle,anomaly_score,anomaly,anomaly_label
0,1139246,66000,999089,3157,121,3,2004,68.0,1,588,...,-1,-1,-1,2,1,2006-11-16,2,0.084238,1,Normal
1,1139248,57000,117657,77,121,3,1996,4640.0,1,1568,...,-1,-1,-1,2,1,2004-03-26,8,0.072997,1,Normal
2,1139249,10000,434808,7009,121,3,2001,2838.0,0,1248,...,-1,-1,-1,-1,-1,2004-02-26,3,0.096354,1,Normal
3,1139251,38500,1026470,332,121,3,2001,3486.0,0,2183,...,-1,-1,-1,-1,-1,2011-05-19,10,0.093114,1,Normal
4,1139253,11000,1057373,17311,121,3,2007,722.0,2,2901,...,-1,-1,-1,-1,-1,2009-07-23,2,0.062077,1,Normal


In [29]:
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

for col in df3.select_dtypes(['object']):
    df3[col] = df2[col].astype('category')
df3.info()

for col in df3.select_dtypes(['category']):
    df3[col] = df3[col].cat.codes
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126919 entries, 0 to 126918
Data columns (total 58 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   126919 non-null  int64         
 1   SalePrice                 126919 non-null  int64         
 2   MachineID                 126919 non-null  int64         
 3   ModelID                   126919 non-null  int64         
 4   datasource                126919 non-null  int64         
 5   auctioneerID              126919 non-null  int64         
 6   YearMade                  126919 non-null  int64         
 7   MachineHoursCurrentMeter  26735 non-null   float64       
 8   UsageBand                 126919 non-null  int8          
 9   saledate                  126919 non-null  int16         
 10  fiModelDesc               126919 non-null  int16         
 11  fiBaseModel               126919 non-null  int16         
 12  fi

In [30]:
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

for col in df3.select_dtypes(['object']):
    df3[col] = df2[col].astype('category')


for col in df3.select_dtypes(['category']):
    df3[col] = df3[col].cat.codes
df3 = df3[df2.anomaly==1].copy()
X2 = df3.drop(columns=['SalePrice', 'anomaly_label', 'anomaly', 'anomaly_score','date_of_sale'])
X2 = pd.get_dummies(X2, drop_first=True)
y2 = df3['SalePrice']
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=42)

model2 = RandomForestRegressor()
model2.fit(X2, y2)
y2_test_pred = model2.predict(X2_test)
y2_train_pred = model2.predict(X2_train)
def RMSE(y2_, y2_pred_):
    return ((y2_ - y2_pred_) ** 2).mean() ** 0.5

print(f'Train RMSE: {RMSE(y2_train, y2_train_pred):.3f}')
print(f'Test RMSE: {RMSE(y2_test, y2_test_pred):.3f}')

Train RMSE: 2687.302
Test RMSE: 2696.150


In [31]:
import joblib
model_data2 = [model2, iso_forest]
joblib.dump(model_data2, '/content/3.2.25_final.pkl')  # Save directly to the file

['/content/3.2.25_final.pkl']

In [95]:
import pandas as pd
df_valid = pd.read_csv("Valid.csv")

In [96]:
print(df_valid.columns)

Index(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID',
       'YearMade', 'MachineHoursCurrentMeter', 'UsageBand', 'saledate',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls'],
      dtype='object')


In [97]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11573 entries, 0 to 11572
Data columns (total 52 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SalesID                   11573 non-null  int64  
 1   MachineID                 11573 non-null  int64  
 2   ModelID                   11573 non-null  int64  
 3   datasource                11573 non-null  int64  
 4   auctioneerID              11573 non-null  int64  
 5   YearMade                  11573 non-null  int64  
 6   MachineHoursCurrentMeter  4739 non-null   float64
 7   UsageBand                 4031 non-null   object 
 8   saledate                  11573 non-null  object 
 9   fiModelDesc               11573 non-null  object 
 10  fiBaseModel               11573 non-null  object 
 11  fiSecondaryDesc           8037 non-null   object 
 12  fiModelSeries             1759 non-null   object 
 13  fiModelDescriptor         2897 non-null   object 
 14  Produc

In [98]:
# Convert 'date_of_sale' to datetime format
df_valid['date1_of_sale'] = pd.to_datetime(df_valid['saledate'], format='%m/%d/%Y %H:%M', errors='coerce')

# Add AgeOfVehicle column (date of sale year - YearMade)
df_valid['AgeOfVehicle'] = df_valid['date1_of_sale'].dt.year - df_valid['YearMade']

# Display the first few rows of the updated DataFrame

df_valid.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,date1_of_sale,AgeOfVehicle
0,1222837,902859,1376,121,3,1000,0.0,NaN,1/5/2012 0:00,375L,...,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN,2012-01-05,1012
1,1222839,1048320,36526,121,3,2006,4412.0,Medium,1/5/2012 0:00,TX300LC2,...,None or Unspecified,Yes,Double,NaN,NaN,NaN,NaN,NaN,2012-01-05,6
2,1222841,999308,4587,121,3,2000,10127.0,Medium,1/5/2012 0:00,270LC,...,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN,2012-01-05,12
3,1222843,1062425,1954,121,3,1000,4682.0,Low,1/5/2012 0:00,892DLC,...,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN,2012-01-05,1012
4,1222845,1032841,4701,121,3,2002,8150.0,Medium,1/4/2012 0:00,544H,...,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,2012-01-04,10


In [94]:
df_valid.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,AgeOfVehicle,anomaly_score,anomaly,date1_of_sale
0,1222837,902859,1376,121,3,1000,0.0,-1,19,293,...,0,-1,-1,-1,-1,-1,NaN,0.161004,1,NaT
1,1222839,1048320,36526,121,3,2006,4412.0,2,19,1611,...,0,-1,-1,-1,-1,-1,NaN,0.129481,1,NaT
2,1222841,999308,4587,121,3,2000,10127.0,2,19,160,...,0,-1,-1,-1,-1,-1,NaN,0.176823,1,NaT
3,1222843,1062425,1954,121,3,1000,4682.0,1,19,686,...,0,-1,-1,-1,-1,-1,NaN,0.138901,1,NaT
4,1222845,1032841,4701,121,3,2002,8150.0,2,18,397,...,-1,-1,-1,-1,2,1,NaN,0.152849,1,NaT


In [99]:
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

for col in df_valid.select_dtypes(['object']):
    df_valid[col] = df_valid[col].astype('category')

for col in df_valid.select_dtypes(['category']):
    df_valid[col] = df_valid[col].cat.codes
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11573 entries, 0 to 11572
Data columns (total 54 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   SalesID                   11573 non-null  int64         
 1   MachineID                 11573 non-null  int64         
 2   ModelID                   11573 non-null  int64         
 3   datasource                11573 non-null  int64         
 4   auctioneerID              11573 non-null  int64         
 5   YearMade                  11573 non-null  int64         
 6   MachineHoursCurrentMeter  4739 non-null   float64       
 7   UsageBand                 11573 non-null  int8          
 8   saledate                  11573 non-null  int8          
 9   fiModelDesc               11573 non-null  int16         
 10  fiBaseModel               11573 non-null  int16         
 11  fiSecondaryDesc           11573 non-null  int8          
 12  fiModelSeries     

In [100]:
print(X2.shape)  # Shape of the data you're passing to decision_function
print(df_valid.shape)  # Shape of the DataFrame you're adding the 'anomaly_score' to

(11573, 53)
(11573, 54)


In [101]:
df_valid=df_valid.drop(columns=['date1_of_sale'])
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11573 entries, 0 to 11572
Data columns (total 53 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SalesID                   11573 non-null  int64  
 1   MachineID                 11573 non-null  int64  
 2   ModelID                   11573 non-null  int64  
 3   datasource                11573 non-null  int64  
 4   auctioneerID              11573 non-null  int64  
 5   YearMade                  11573 non-null  int64  
 6   MachineHoursCurrentMeter  4739 non-null   float64
 7   UsageBand                 11573 non-null  int8   
 8   saledate                  11573 non-null  int8   
 9   fiModelDesc               11573 non-null  int16  
 10  fiBaseModel               11573 non-null  int16  
 11  fiSecondaryDesc           11573 non-null  int8   
 12  fiModelSeries             11573 non-null  int8   
 13  fiModelDescriptor         11573 non-null  int8   
 14  Produc

In [111]:
X2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11573 entries, 0 to 12100
Data columns (total 53 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SalesID                   11573 non-null  int64  
 1   MachineID                 11573 non-null  int64  
 2   ModelID                   11573 non-null  int64  
 3   datasource                11573 non-null  int64  
 4   auctioneerID              11573 non-null  int64  
 5   YearMade                  11573 non-null  int64  
 6   MachineHoursCurrentMeter  11573 non-null  float64
 7   UsageBand                 11573 non-null  int8   
 8   saledate                  11573 non-null  int16  
 9   fiModelDesc               11573 non-null  int16  
 10  fiBaseModel               11573 non-null  int16  
 11  fiSecondaryDesc           11573 non-null  int8   
 12  fiModelSeries             11573 non-null  int8   
 13  fiModelDescriptor         11573 non-null  int8   
 14  ProductSize

In [102]:
print(X2.shape)  # Shape of the data you're passing to decision_function
print(df_valid.shape)  # Shape of the DataFrame you're adding the 'anomaly_score' to

(11573, 53)
(11573, 53)


In [112]:
print(df_valid.columns)  # List all column names in df_valid
print(X2.columns)  # List all column names in X2


Index(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID',
       'YearMade', 'MachineHoursCurrentMeter', 'UsageBand', 'saledate',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'AgeOfVehicle',
       'anomaly_score'],
      dtype='object')
Index(['SalesID', 'MachineI

In [113]:
print(X2.shape)  # Shape of the data you're passing to decision_function
print(df_valid.shape)  # Shape of the DataFrame you're adding the 'anomaly_score' to

(11573, 53)
(11573, 54)


In [128]:
df_valid = df_valid.drop(columns=['anomaly_score'], errors='ignore')


In [129]:
print(X2.shape)  # Shape of the data you're passing to decision_function
print(df_valid.shape)  # Shape of the DataFrame you're adding the 'anomaly_score' to

(11573, 53)
(11573, 53)


In [130]:
print(df_valid.columns)  # List all column names in df_valid
print(X2.columns)  # List all column names in X2

Index(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID',
       'YearMade', 'MachineHoursCurrentMeter', 'UsageBand', 'saledate',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'AgeOfVehicle'],
      dtype='object')
Index(['SalesID', 'MachineID', 'ModelID', 'datasour

In [131]:
print(X2.shape)  # Shape of the data you're passing to decision_function
print(df_valid.shape)  # Shape of the DataFrame you're adding the 'anomaly_score' to

(11573, 53)
(11573, 53)


In [125]:
#df_valid = df_valid[X2.columns]

In [126]:

# Reset the index to avoid any index-related issues
#f_valid = df_valid.reset_index(drop=True)

In [133]:
# Now, compute the anomaly scores and labels

df_valid['anomaly_score'] = iso_forest.decision_function(df_valid)  # Quantitative weirdness
df_valid['anomaly'] = iso_forest.predict(df_valid)  # Binary anomaly label
df_valid['anomaly_label'] = df_valid['anomaly'].map({1: 'Normal', -1: 'Anomaly'})  # Map labels


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- anomaly_score
